# LNDL - Cognitive Programming Language for LLM Structured Output

LNDL (Lion Natural Declaration Language) is a cognitive programming language that enables LLMs to:
- **Think naturally** with prose, reasoning, and revisions
- **Declare structured data** via explicit field mapping
- **Execute actions lazily** (only when referenced in output)
- **Control execution flow** with yields and continuations

This notebook covers:
1. LNDL syntax and semantics
2. Parsing LNDL responses
3. Using LNDL with lionpride operations
4. ReAct loops with tool calling
5. LNDL v2 cognitive extensions

In [ ]:
# Core imports
from pydantic import BaseModel, Field

from lionpride.lndl import (
    Lexer,
    Parser,
    Program,
    get_lndl_system_prompt,
    parse_lndl,
    parse_lndl_fuzzy,
)
from lionpride.types import Operable, Spec

## 1. LNDL Syntax Overview

LNDL allows LLMs to intermix natural reasoning with structured declarations.

### Core Constructs

| Construct | Syntax | Purpose |
|-----------|--------|--------|
| Variable | `<lvar Model.field alias>value</lvar>` | Declare structured data |
| Raw Variable | `<lvar alias>value</lvar>` | Simple string capture |
| Action | `<lact name>function(args)</lact>` | Tool/function call |
| Output | `OUT{field:[vars], scalar:literal}` | Final structured result |

In [ ]:
# Get the LNDL system prompt (sent to LLM as guidance)
system_prompt = get_lndl_system_prompt()
print(f"System prompt: {len(system_prompt)} chars, {len(system_prompt.splitlines())} lines")
print("\n--- First 500 chars ---")
print(system_prompt[:500])

## 2. LNDL Variables - Explicit Field Mapping

Variables use `Model.field` namespacing for explicit mapping to Pydantic models.

In [ ]:
# Example LNDL response from an LLM
lndl_response = """
Let me analyze this step by step.

First, I'll create a title:
<lvar Report.title t1>AI Safety Overview</lvar>

Actually, let me be more specific:
<lvar Report.title t2>Comprehensive AI Safety Analysis for 2025</lvar>

Now for the summary:
<lvar Report.summary s>This report examines current AI safety practices,
identifying key challenges and opportunities for improvement.</lvar>

And a confidence score:
<lvar Report.confidence c>0.85</lvar>

Final output (using refined title t2):
```lndl
OUT{report:[t2, s, c]}
```
"""

print("LNDL Response Example:")
print(lndl_response)

In [ ]:
# Parse the LNDL response
lexer = Lexer(lndl_response)
tokens = lexer.tokenize()
parser = Parser(tokens, source_text=lndl_response)
program: Program = parser.parse()

print("Parsed LNDL Program:")
print(f"  Variables (lvars): {len(program.lvars)}")
for lvar in program.lvars:
    print(f"    - {lvar.model}.{lvar.field} as '{lvar.alias}': {lvar.content[:50]}...")

print(f"\n  Actions (lacts): {len(program.lacts)}")
print(f"\n  Output block: {program.out_block}")

## 3. LNDL Actions - Lazy Execution

Actions are **only executed if referenced in OUT{}** - unreferenced actions are scratch work.

In [ ]:
# Example with actions
lndl_with_actions = """
Let me search for relevant information:

<lact broad>search(query="AI", limit=1000)</lact>
That's too broad. Let me narrow it:

<lact focused>search(query="AI safety research 2025", limit=20)</lact>
Better! Now I'll analyze:

<lvar Report.title t>AI Safety Research Summary</lvar>
<lvar Report.summary s>Based on search results...</lvar>

```lndl
OUT{report:[t, s], data:[focused]}
```
"""

lexer = Lexer(lndl_with_actions)
parser = Parser(lexer.tokenize(), source_text=lndl_with_actions)
program = parser.parse()

print("Actions declared:")
for lact in program.lacts:
    print(f"  - {lact.alias}: {lact.call}")

print(f"\nOUT block references: {program.out_block.fields if program.out_block else 'None'}")
print("\nExecution behavior:")
print("  - 'focused' EXECUTES (in OUT{})")
print("  - 'broad' NOT executed (scratch work)")

## 4. Resolving LNDL to Pydantic Models

Use `parse_lndl` to resolve LNDL output to validated Pydantic models.

In [ ]:
# Define the target Pydantic model
class Report(BaseModel):
    """Analysis report model."""

    title: str = Field(..., description="Report title")
    summary: str = Field(..., description="Report summary")
    confidence: float = Field(default=0.5, ge=0.0, le=1.0)


# Create an Operable to define the expected output structure
report_operable = Operable(
    specs=[
        Spec(Report, name="report"),
    ],
    name="ReportOutput",
)

print(f"Operable: {report_operable.name}")
print(f"Specs: {[s.name for s in report_operable.get_specs()]}")

In [ ]:
# Parse LNDL and resolve to model
lndl_simple = """
<lvar Report.title t>AI Safety in Modern Systems</lvar>
<lvar Report.summary s>This report provides an overview of AI safety practices.</lvar>
<lvar Report.confidence c>0.9</lvar>

OUT{report:[t, s, c]}
"""

try:
    result = parse_lndl(lndl_simple, report_operable)
    print("Parsed result:")
    print(f"  Type: {type(result)}")
    print(f"  Fields: {result.model_fields.keys() if hasattr(result, 'model_fields') else result}")
    if hasattr(result, "report"):
        print(f"\n  report.title: {result.report.title}")
        print(f"  report.summary: {result.report.summary[:50]}...")
        print(f"  report.confidence: {result.report.confidence}")
except Exception as e:
    print(f"Parse error: {e}")

## 5. Fuzzy LNDL Parsing

For LLM outputs that may have minor syntax issues, use fuzzy parsing.

In [ ]:
# Fuzzy parsing handles imperfect LLM output
messy_lndl = """
Here's my analysis:

<lvar Report.title t>Safety Analysis</lvar>
<lvar Report.summary s>The analysis shows significant progress in AI safety.</lvar>

OUT{report:[t, s]}
"""

# Fuzzy parse with threshold (0.0-1.0)
fuzzy_result = parse_lndl_fuzzy(messy_lndl, report_operable, threshold=0.7)

print("Fuzzy parse result:")
print(f"  Success: {fuzzy_result.success}")
print(f"  Confidence: {fuzzy_result.confidence:.2f}")
if fuzzy_result.data:
    print(f"  Data type: {type(fuzzy_result.data)}")

## 6. Using LNDL with lionpride Operations

The `communicate` and `operate` functions support LNDL via the `operable` parameter.

In [ ]:
# NOTE: This example requires API keys and won't run without them
# It demonstrates the correct API usage pattern


# Setup (requires OPENAI_API_KEY or similar)
# session = Session()
# model = iModel(provider="openai", model="gpt-4o-mini")
# session.services.register(model)
# branch = session.create_branch(name="analysis")

print("API Pattern for LNDL with communicate():")
print("""
# Using LNDL mode via operable parameter
result = await communicate(
    session=session,
    branch=branch,
    parameters=CommunicateParam(
        instruction="Analyze the current state of AI safety research",
        imodel=model.name,
        operable=report_operable,  # Enables LNDL mode
        return_as="model",
        lndl_threshold=0.7,  # Fuzzy matching threshold
    )
)
""")

In [ ]:
print("API Pattern for LNDL with operate():")
print("""
# Using operate with LNDL mode
result = await operate(
    session=session,
    branch=branch,
    parameters=OperateParam(
        instruction="Analyze AI safety and provide reasoning",
        imodel=model.name,
        operable=report_operable,  # LNDL structured output
        use_lndl=True,
        reason=True,  # Add reasoning field
        actions=False,  # No tool calling in this example
    )
)
""")

## 7. ReAct Pattern - Multi-Step Tool Calling

The `react` operation implements the ReAct (Reasoning + Acting) pattern for multi-step tasks.

In [ ]:
from lionpride.services.types import Tool


# Define a tool
def search_documents(query: str, limit: int = 10) -> list[dict]:
    """Search for documents matching the query.

    Args:
        query: Search query string
        limit: Maximum results to return
    """
    # Mock implementation
    return [
        {"title": f"Result {i}", "snippet": f"Content for {query}"} for i in range(min(limit, 3))
    ]


# Create Tool instance
search_tool = Tool(func_callable=search_documents)

print(f"Tool: {search_tool.name}")
print(f"Schema: {search_tool.tool_schema}")

In [ ]:
print("API Pattern for react():")
print("""
# ReAct loop with tool calling
result = await react(
    session=session,
    branch=branch,
    parameters=ReactParam(
        instruction="Find information about AI safety best practices",
        imodel=model.name,
        model_name=model.name,
        tools=[search_tool],  # Available tools
        max_steps=5,  # Maximum reasoning steps
        response_model=Report,  # Final answer structure
        use_lndl=True,
        verbose=True,  # Show step-by-step execution
    )
)

# Result contains:
# - result.steps: List of ReactStep with reasoning, actions, observations
# - result.final_response: Validated final answer
# - result.completed: Whether task completed successfully
# - result.total_steps: Number of steps taken
""")

## 8. LNDL v2 Cognitive Extensions

LNDL v2 adds cognitive programming features for advanced workflows.

In [ ]:
print("LNDL v2 Cognitive Extensions:")
print("""
1. CONTEXT MANAGEMENT
   <context>
     <include msg="msg_id"/>           # Include specific message
     <compress msgs="0..50" to="ctx"/> # Compress messages to symbol
     <drop msg="verbose_output"/>      # Drop from context
     <notice msg="tools" brief="..."/> # Brief notice
   </context>

2. CONTINUATION CONTROL
   <yield for="action" reason="need results" keep="top_5" drop_full="true"/>
   
   - Pause execution to wait for action results
   - Transform observations (keep, drop_full, transform)
   - Resume with observation injected

3. MULTI-AGENT COMMUNICATION
   <send to="critic" type="ReviewRequest" timeout="30s">
     <include msg="analysis"/>
   </send>
""")

In [ ]:
# Example LNDL v2 response with yield
lndl_v2_example = """
I need to search for data first.

<lact search>search(query="AI safety", limit=10)</lact>

<yield for="search" reason="need search results" keep="top_3"/>

Based on the results, here's my analysis:
<lvar Report.title t>AI Safety Analysis</lvar>
<lvar Report.summary s>The search results show...</lvar>

OUT{report:[t, s]}
"""

# Parse v2 LNDL
lexer = Lexer(lndl_v2_example)
parser = Parser(lexer.tokenize(), source_text=lndl_v2_example)
program = parser.parse()

print("LNDL v2 Program:")
print(f"  Yields: {len(program.yields) if program.yields else 0}")
if program.yields:
    for y in program.yields:
        print(f"    - for: {y.for_ref}, reason: {y.reason}, keep: {y.keep}")

## 9. Cognitive React - Full Cognitive Loop

The `cognitive_react` function implements the full LNDL v2 cognitive loop.

In [ ]:
print("API Pattern for cognitive_react():")
print("""
# Define permissions for cognitive operations
permissions = CognitivePermission(
    can_include=True,   # Can include specific messages
    can_compress=True,  # Can request message compression
    can_drop=True,      # Can request message dropping
    can_yield=True,     # Can yield for continuation
    max_yields=10,      # Maximum yields per session
    can_send=False,     # Multi-agent (disabled)
)

# Action executor for tool calls
async def execute_action(name, call, observations):
    # Execute the action and return result
    if name == "search":
        return {"results": ["result1", "result2"]}
    return None

# Run cognitive react loop
result = await cognitive_react(
    session=session,
    branch=branch,
    instruction="Research AI safety and provide a comprehensive report",
    imodel=model.name,
    action_executor=execute_action,
    permissions=permissions,
    max_iterations=5,
    verbose=True,
)

# Result structure:
# - result.phases: List of execution phases
# - result.final_output: Parsed OUT{} block data
# - result.observations: Accumulated action results
# - result.completed: Whether completed successfully
""")

## 10. TypeScript Schema for Input Efficiency

lionpride uses TypeScript notation for compact schema representation in prompts.

In [ ]:
import json

from lionpride.libs.schema_handlers import typescript_schema

# Get JSON Schema from Pydantic model
json_schema = Report.model_json_schema()

# Convert to TypeScript notation
ts_schema = typescript_schema(json_schema)

print("JSON Schema (verbose):")
print(json.dumps(json_schema, indent=2))
print(f"\nJSON Schema tokens: ~{len(json.dumps(json_schema)) // 4}")

print("\n" + "=" * 50)
print("\nTypeScript Schema (compact):")
print(ts_schema)
print(f"\nTypeScript tokens: ~{len(ts_schema) // 4}")

In [ ]:
# Complex model example
class Employment(BaseModel):
    company: str
    position: str
    start_date: str
    end_date: str | None = None


class Person(BaseModel):
    name: str
    age: int
    email: str
    current_job: Employment
    previous_jobs: list[Employment] = []
    skills: list[str] = []
    is_active: bool = True


json_schema = Person.model_json_schema()
ts_schema = typescript_schema(json_schema)

print("Complex Model - TypeScript Schema:")
print(ts_schema)
print(f"\nJSON: ~{len(json.dumps(json_schema)) // 4} tokens")
print(f"TypeScript: ~{len(ts_schema) // 4} tokens")
print(f"Reduction: ~{(1 - len(ts_schema) / len(json.dumps(json_schema))) * 100:.0f}%")

## Summary

### LNDL Syntax
- **Variables**: `<lvar Model.field alias>value</lvar>` - explicit field mapping
- **Actions**: `<lact name>function(args)</lact>` - lazy execution
- **Output**: `OUT{field:[vars], scalar:literal}` - final structure

### Key Features
- **Natural thinking**: Prose + declarations intermixed
- **Lazy execution**: Only OUT{}-referenced actions run
- **Multiple versions**: Explore options, select final in OUT{}
- **Fuzzy parsing**: Tolerates minor LLM syntax errors

### Integration Points
- `communicate(operable=...)` - LNDL mode for structured chat
- `operate(use_lndl=True)` - LNDL with reasoning/actions
- `react(...)` - Multi-step ReAct loops
- `cognitive_react(...)` - Full LNDL v2 cognitive programming

### LNDL v2 Extensions
- Context management (`<context>`, `<compress>`, `<drop>`)
- Continuation control (`<yield>` for action results)
- Multi-agent communication (`<send>` to other agents)